In [1]:
import pandas as pd

data =  pd.read_csv(r'yelp_academic_dataset_review.csv',encoding = "utf8",keep_default_na=False,nrows=200000)


from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)
data.to_sql('yelp_reviews',con=engine)


In [2]:
data.head()

,user_id,review_id,text,votes.cool,business_id,votes.funny,stars,date,type,votes.useful
0,Xqd0DzHaiyRqVH3WRG7hzg,15SdjuK7DmYqUAj6rjGowg,dr. goldberg offers everything i look for in a...,1,vcNAWiLM4dR7D2nwwJ7nCA,0,5,2007-05-17,review,2
1,H1kH6QZV7Le4zqTRNxoZow,RF6UnRTtG7tWMcrO2GEoAg,"Unfortunately, the frustration of being Dr. Go...",0,vcNAWiLM4dR7D2nwwJ7nCA,0,2,2010-03-22,review,2
2,zvJCcrpm2yOZrxKffwGQLA,-TsVN230RCkLYKBeLsuz7A,Dr. Goldberg has been my doctor for years and ...,1,vcNAWiLM4dR7D2nwwJ7nCA,0,4,2012-02-14,review,1
3,KBLW4wJA_fwoWmMhiHRVOA,dNocEAyUucjT371NNND41Q,Been going to Dr. Goldberg for over 10 years. ...,0,vcNAWiLM4dR7D2nwwJ7nCA,0,4,2012-03-02,review,0
4,zvJCcrpm2yOZrxKffwGQLA,ebcN2aqmNUuYNoyvQErgnA,Got a letter in the mail last week that said D...,1,vcNAWiLM4dR7D2nwwJ7nCA,0,4,2012-05-15,review,2


In [3]:
data.describe()

,votes.cool,votes.funny,stars,votes.useful
count,200000.000000,200000.000000,200000.000000,200000.000000
mean,0.644440,0.534925,3.660460,1.124985
std,1.609059,1.614179,1.334991,1.962349
min,0.000000,0.000000,1.000000,0.000000
25%,0.000000,0.000000,3.000000,0.000000
50%,0.000000,0.000000,4.000000,1.000000
75%,1.000000,0.000000,5.000000,1.000000
max,65.000000,68.000000,5.000000,67.000000


In [4]:
data.shape

(200000, 10)

In [5]:
reviews = data[:45000]["text"]
reviews.head()

0    dr. goldberg offers everything i look for in a...
1    Unfortunately, the frustration of being Dr. Go...
2    Dr. Goldberg has been my doctor for years and ...
3    Been going to Dr. Goldberg for over 10 years. ...
4    Got a letter in the mail last week that said D...
Name: text, dtype: object

In [6]:
import string
from nltk.corpus import stopwords

def tokenize_reviews(review):
    rev = [char for char in review if char not in string.punctuation]
    rev = ''.join(rev)
    
    return [word for word in rev.split() if word.lower() not in stopwords.words('english')]

In [7]:
print(tokenize_reviews(reviews[16]))

['Ate', 'Saturday', 'morning', 'breakfast', 'Pine', 'Cone', 'Friendlyquick', 'service', 'normal', 'prices', 'new', 'year', 'special', 'allyoucaneat', 'pancakes', '212', 'added', 'breakfast', 'sausage', 'patty', '300', 'Father', 'typical', 'two', 'eggs', 'toast', 'hashbowns', 'seem', 'fresh', 'cut', 'sausage', 'links', 'Food', 'tasty', 'major', 'qualms', 'hashbrowns', 'need', 'bit', 'cooking', 'coffee', 'needs', 'bit', 'bite', 'nice', 'assortment', 'bakes', 'goods', 'massive', 'large', 'head', 'creampuffs', 'eclairs', 'took', 'blueberry', 'muffin', 'go', 'enjoyed', 'Quick', 'exit', 'I94just', 'northeast', 'Madison']


In [8]:
review_tokens = []
dataset = []
for review in reviews:
    if len(review) > 10:
        review_tokens.append(tokenize_reviews(review))
        dataset.append(review)

In [9]:
print(review_tokens[44000])

['gem', 'location', 'Ahwatukee', '11', 'years', 'next', 'Va', 'Bene', 'cant', 'believe', 'found', 'Italian', 'Im', 'pretty', 'picky', 'Italian', 'food', 'place', 'good', 'reasonably', 'priced', 'outset', 'though', 'doubts', 'bread', 'really', 'poor', 'excuse', 'Italian', 'bread', 'soft', 'anything', 'like', 'crusty', 'Italian', 'bread', 'Im', 'used', 'Normally', 'bode', 'well', 'quality', 'food', 'However', 'Lasagna', 'eggplant', 'parm', 'good', 'dont', 'think', 'Ill', 'ever', 'find', 'good', 'mamas', 'Salad', 'house', 'Italian', 'dressing', 'quite', 'good', 'wilted', 'lettuce', 'another', 'reviewer', 'experienced', 'Trio', 'pizza', 'ricotta', 'tomato', 'basil', 'clam', 'pizza', 'excellent', 'Never', 'clam', 'pizza', 'garlicky', 'wonderful', 'also', 'made', 'red', 'clam', 'sauce', 'version', 'good', 'garlicky', 'white', 'Also', 'pasta', 'broccoli', 'sausage', 'Excellent', 'thing', 'clam', 'pizza', 'pasta', 'bit', 'oily', 'liking', 'Next', 'time', 'Ill', 'ask', 'cut', 'back', 'oil', 'ma

In [10]:
file = open(r'positive-words.txt')
positive  = file.read()
positive = positive.lower()
file = open(r'negative-words.txt')
negative  = file.read()

In [11]:
def assign_score(token):
    score = []
    for word in token:
        if word in positive:
            score.append(1)
        elif word in negative:
            score.append(-1)
        else:
            score.append(0)
    return score

In [12]:
review_scores = []
for token in review_tokens:
    review_scores.append(assign_score(token))

In [13]:
from statistics import mean
labels = []
pos = neg = neut = 0         
for score in review_scores:
    if score:
        val =  mean(score)
        if val > 0.29:
            labels.append(1)
            pos += 1
        elif 0.212 <= val <= 0.29:
            labels.append(0)
            neut += 1
        else:
            labels.append(-1)
            neg += 1
    else:
        labels.append(0)

print(pos,neg,neut)

13999 14922 16033


In [14]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(dataset,labels,test_size=0.25,random_state=108)

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsOneClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer

nb = Pipeline([('vect', CountVectorizer(analyzer = 'word',lowercase = True,stop_words='english')),
               ('tfidf', TfidfTransformer(smooth_idf=True)),
               ('clf', LogisticRegression(penalty='l2',solver='newton-cg',multi_class='multinomial')),
              ])
nb.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        ...ty='l2', random_state=None, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False))])

In [16]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(nb, x_train,y_train , cv=5)
print(scores)

[0.6483321  0.66750704 0.63932968 0.65514684 0.65722338]


In [17]:
y_pred = nb.predict(x_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.6552184358038972

In [18]:
y_pred = []
y_pred = nb.predict(x_test)
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          -1       0.69      0.71      0.70      3761
           0       0.56      0.55      0.56      4022
           1       0.72      0.71      0.72      3456

   micro avg       0.66      0.66      0.66     11239
   macro avg       0.66      0.66      0.66     11239
weighted avg       0.65      0.66      0.65     11239



In [19]:
import heapq
counter = []
counter = engine.execute("SELECT COUNT(*) AS count,business_id FROM yelp_reviews GROUP BY business_id").fetchall()
rest = heapq.nlargest(10,counter)

In [20]:
results = []
correct = 0
for count,id in rest:
    data_base = []
    stars = []
    prediction = ""
    data_base = engine.execute("SELECT text from yelp_reviews where business_id = ?",(id)).fetchall()
    stars = engine.execute("SELECT stars as INTEGER from yelp_reviews where business_id = ?",(id)).fetchall()
    new_db = []
    stars_db = []
    for data in data_base:
        new_db.append(str(data))

    for star in stars:
        stars_db.append(star[0])
        
    ypred = nb.predict(new_db)
    y_pred_new = []
    for pred in ypred:
        if pred == 0:
            y_pred_new.append(3.5)
        elif pred == -1:
            y_pred_new.append(1.5)
        else:
            y_pred_new.append(5)
        
#         print(y_pred_new)
    predicted = mean(y_pred_new)
    actual = mean(stars_db)
    if abs(predicted - actual) < 0.7:
        prediction = "correct"
        correct += 1
    else:
        prediction = "wrong"
    results.append([predicted,actual,id,prediction])

In [22]:
from tabulate import tabulate
print(tabulate(results,headers=['Predicted', 'Actual', 'Business_id','Result'], tablefmt='orgtbl'))

|   Predicted |   Actual | Business_id            | Result   |
|-------------+----------+------------------------+----------|
|     2.74692 |  3.27635 | jf67Z1pnwElRSXllpQHiJg | correct  |
|     3.43436 |  3.47659 | FV16IeXJp2W6pnghTz2FAw | correct  |
|     2.82236 |  3.24557 | hW0Ne_HTHEAgGF1rAdmR-g | correct  |
|     3.37163 |  4.31154 | JokKtdXU7zXHcr20Lrk29A | wrong    |
|     3.19733 |  4.06886 | 4UVhuOLaMm2-34SrW8y-ag | wrong    |
|     3.31265 |  4.48724 | SDwYQ6eSu1htn8vHWv128g | wrong    |
|     3.19167 |  4.18182 | K8pM6qQdYu5h6buRE1-_sw | wrong    |
|     2.68302 |  2.51402 | KJnVuzpveyDrHARVNZaYVg | correct  |
|     3.04538 |  4.33168 | c1yGkETheht_1vjda7G5sA | wrong    |
|     3.27787 |  4.27959 | L9UYbtAUOcfTgZFimehlXw | wrong    |
